In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
df = pd.read_csv('/Users/dyadyundanil/Desktop/Machine learning/DATA/Advertising.csv')

In [5]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [7]:
#cross_val_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import root_mean_squared_error

X = df.drop('sales', axis=1)
y = df['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

best_alpha = -1
best_metrics = float('inf')
alphas = [0.1, 1, 10, 20, 100]
for alpha in alphas:
    model = Ridge(alpha=alpha)
    results = cross_val_score(model, X_train_scaled, y_train, scoring='neg_root_mean_squared_error', cv=5)
    print('Results for each k train-test fit:', results)
    if abs(results.mean()) < best_metrics:
        best_metrics = abs(results.mean())
        best_alpha = alpha
        
print('the best alpha:', best_alpha)

Results for each k train-test fit: [-2.10945128 -1.48269544 -1.36608846 -1.64092812 -1.98539629]
Results for each k train-test fit: [-2.10463498 -1.48590122 -1.36843344 -1.64766931 -1.98144225]
Results for each k train-test fit: [-2.07884806 -1.54702059 -1.42252488 -1.73400892 -1.96535827]
Results for each k train-test fit: [-2.08768688 -1.65432653 -1.52638634 -1.85551867 -1.98513535]
Results for each k train-test fit: [-2.59490468 -2.58533906 -2.49961657 -2.82668978 -2.50762916]
the best alpha: 0.1


In [9]:
#cross_validate
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import root_mean_squared_error

X = df.drop('sales', axis=1)
y = df['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


model = Ridge(alpha=0.1)
scoring = ['neg_root_mean_squared_error', 'neg_mean_squared_error']
results = cross_validate(model, X_train_scaled, y_train, scoring=scoring, cv=5)
pd.DataFrame(results)


,fit_time,score_time,test_neg_root_mean_squared_error,test_neg_mean_squared_error
0,0.001971,0.001982,-2.109451,-4.449785
1,0.001458,0.001218,-1.482695,-2.198386
2,0.000846,0.000557,-1.366088,-1.866198
3,0.000585,0.000480,-1.640928,-2.692645
4,0.000631,0.000493,-1.985396,-3.941798


In [11]:
#Чтобы не писать собственно ручно циклы для отыскания наилучших гиперпараметров модели применяется ф-ция GridSearchCV
#На каждой итерации (определенном наборе гиперпараметров) оценивается средняя ошибка с помощью кросс-валидации
#И в конце происходит сравнение всех этих ошибок и выбор наименьшей -> наилучшие гиперпараметры

In [25]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X = df.drop('sales', axis=1)
y = df['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = ElasticNet()
param_grid = {'alpha':[0.1, 0.5, 1, 10], 'l1_ratio':[0.1, 0.2, 0.3, 0.9]}
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    scoring='neg_root_mean_squared_error',
    verbose=0
)

grid_search.fit(X_train_scaled, y_train)

print('best parameters:', grid_search.best_params_)
print('best estimator:', grid_search.best_estimator_)
print('best score:', grid_search.best_score_)

best parameters: {'alpha': 0.1, 'l1_ratio': 0.9}
best estimator: ElasticNet(alpha=0.1, l1_ratio=0.9)
best score: -1.7183844606988488
